In [28]:
import pandas as pd
from ipywidgets import widgets
from IPython.display import display
pd.options.display.max_colwidth = 10000
pd.options.display.max_rows = 10000

all_payments = pd.read_csv("/Users/johnclark/Documents/Finances/All_Payment_Methods020122.csv")
end_of_year = pd.read_csv("/Users/johnclark/Documents/Finances/Year End All Payment Methods020122.csv",parse_dates=[2], header=0)
end_of_year["Amount_value"] = end_of_year["Amount"].apply(lambda x: float(x[1:].replace(",","")) if x.startswith("$") else -float(x[2:].replace(",","")) if x.startswith("-$") else x)

no_credit_card = end_of_year[end_of_year["Payee"] != "WELLS FARGO CREDIT CARD"]
no_wells_credit_rec_trans = end_of_year[(end_of_year["Payee"] != "WELLS FARGO CREDIT CARD") & (~end_of_year["Description"].str.startswith("RECURRING TRANSFER")) \
                                        & (~end_of_year["Description"].str.startswith("ONLINE TRANSFER TO CLARK J SAVINGS XXXXXX5751")) \
                                        & (~end_of_year["Description"].str.startswith("ONLINE TRANSFER TO CLARK J CHECKING XXXXXX9873"))]

year = 2021
utility_check = pd.read_csv("/Users/johnclark/Documents/Finances/Utility{}.csv".format(year), header=None, names=["Date","Amount","Unk","CheckNum","Description"], parse_dates=[0])
utility_check["account_name"] = "utility"
umbrella_check = pd.read_csv("/Users/johnclark/Documents/Finances/Umbrella{}.csv".format(year), header=None, names=["Date","Amount","Unk","CheckNum","Description"], parse_dates=[0])
umbrella_check["account_name"] = "umbrella"
savings_check = pd.read_csv("/Users/johnclark/Documents/Finances/Savings{}.csv".format(year), header=None, names=["Date","Amount","Unk","CheckNum","Description"], parse_dates=[0])
savings_check["account_name"] = "savings"
primary_check = pd.read_csv("/Users/johnclark/Documents/Finances/Primary{}.csv".format(year), header=None, names=["Date","Amount","Unk","CheckNum","Description"], parse_dates=[0])
primary_check["account_name"] = "primary"
# platinum_card = pd.read_csv("/Users/johnclark/Documents/Finances/PlatinumCard{}.csv".format(year), header=None, names=["Date","Amount","Unk","CheckNum","Description"], parse_dates=[0])
# platinum_card["account_name"] = "platinum"
# cashwise_card = pd.read_csv("/Users/johnclark/Documents/Finances/CashWiseCard{}.csv".format(year), header=None, names=["Date","Amount","Unk","CheckNum","Description"], parse_dates=[0])
# cashwise_card["account_name"] = "cashwise"

accounts_joined = utility_check.append(umbrella_check)
accounts_joined = accounts_joined.append(savings_check)
accounts_joined = accounts_joined.append(primary_check)
# accounts_joined = accounts_joined.append(platinum_card)
# accounts_joined = accounts_joined.append(cashwise_card)

assert len(accounts_joined) == len(utility_check) + len(umbrella_check) + len(savings_check) + len(primary_check)# + len(platinum_card) + len(cashwise_card)

accounts_joined["spending"] = accounts_joined["Amount"].apply(lambda x: x if x<0.0 else 0.0)
accounts_joined["income"] = accounts_joined["Amount"].apply(lambda x: x if x>0.0 else 0.0)

exclude_desc = {"FID BKG SVC LLC MONEYLINE 210402 X49800767 O0TLA JOHN P CLARK","RECURRING TRANSFER"}
tithe_desc_starts = {"RURAL INDIA MINI", "BILL PAY Church of the Open Door",
                     "CHECK # 136","BUSINESS TO BUSINESS ACH www.gracepres.co", 
                     "CHECK # 198",
                     "BILL PAY Church of the Resurrection",
                     "Flagstaff Christ"}
tax_desc_starts = {"CHECK # 132",
                   "CHECK # 131",
                   "FRANCHISE TAX BD CASTTAXRFD"}

accounts_joined["chase"] = accounts_joined.apply(lambda r: r["Amount"] if r["Description"].startswith("CHASE") else 0.0, raw=False, axis=1)
accounts_joined["tithe"] = accounts_joined.apply(lambda r: r["Amount"] if len([1 for td in tithe_desc_starts if r["Description"].startswith(td)]) > 0  else 0.0, raw=False, axis=1)
accounts_joined["taxes"] = accounts_joined.apply(lambda r: r["Amount"] if len([1 for td in tax_desc_starts if r["Description"].startswith(td)]) > 0  else 0.0, raw=False, axis=1)
accounts_joined["taxes_spend"] = accounts_joined.apply(lambda r: r["Amount"] if len([1 for td in tax_desc_starts if r["Description"].startswith(td)]) > 0 and r["Amount"] < 0.0  else 0.0, raw=False, axis=1)

accounts_joined["filtered_amount"] = accounts_joined.apply(lambda r: r["Amount"] if not r["Description"] in exclude_desc else 0.0, raw=False, axis=1)
accounts_joined["filtered_income"] = accounts_joined.apply(lambda r: r["Amount"] if r["Amount"]>0.0 and not r["Description"] in exclude_desc else 0.0, raw=False, axis=1)

accounts_joined["filtered_spending"] = accounts_joined.apply(lambda r: r["Amount"] if r["Amount"]<0.0 and len([1 for td in exclude_desc if r["Description"].startswith(td)]) == 0 else 0.0, raw=False, axis=1)
accounts_joined["filtered_spending"] = accounts_joined["filtered_spending"] - accounts_joined["tithe"] - accounts_joined["taxes_spend"]



In [29]:
columns = ["Amount", "spending","income","filtered_amount","filtered_spending","filtered_income","chase","tithe","taxes","taxes_spend"]
sum_table = accounts_joined.groupby(pd.Grouper(key='Date', freq='1M')).sum("Amount")[columns]
sum_table.loc["Totals"] = sum_table[columns].sum()
display(sum_table)
#display(accounts_joined["Amount"].sum())


,Amount,spending,income,filtered_amount,filtered_spending,filtered_income,chase,tithe,taxes,taxes_spend
Date,,,,,,,,,,
2021-01-31 00:00:00,-3262.54,-16030.75,12768.21,-3262.54,-8728.78,12768.21,-5536.14,-2800.25,0.0,0.0
2021-02-28 00:00:00,-9186.00,-22079.10,12893.10,-9186.00,-14677.13,12893.10,-7927.58,-2800.25,0.0,0.0
2021-03-31 00:00:00,-6911.47,-39904.59,32993.12,-6911.47,-22402.62,32993.12,-5281.05,-12800.25,0.0,0.0
2021-04-30 00:00:00,66655.41,-35636.79,102292.20,16655.41,-8311.82,52292.20,-4580.39,-2800.25,2469.0,-19923.0
2021-05-31 00:00:00,304.59,-13254.76,13559.35,304.59,-5952.79,13559.35,-3412.34,-2800.25,0.0,0.0
2021-06-30 00:00:00,-1369.97,-15697.55,14327.58,-1369.97,-8195.58,14327.58,-4841.89,-2800.25,0.0,0.0
2021-07-31 00:00:00,-6443.26,-67095.63,60652.37,-6443.26,-54793.66,60652.37,-6998.29,-7800.25,0.0,0.0
2021-08-31 00:00:00,-5549.34,-19310.10,13760.76,-5549.34,-11808.13,13760.76,-8118.73,-2800.25,0.0,0.0
2021-09-30 00:00:00,-8070.01,-21731.82,13661.81,-8070.01,-9329.85,13661.81,-5999.85,-7800.25,0.0,0.0


## Show month

In [21]:

month = 3

start_date = '2021-{}-01'.format(month)
if month < 12:
    end_date = '2021-{}-01'.format(month+1)
else:
    end_date = '2022-01-01'
view_table = accounts_joined[(accounts_joined["Date"] >= start_date) & (accounts_joined["Date"] < end_date)].sort_values("Date")
view_table["comment"] = ""

notes = {"CHECK # 132":"Arizona Department of Revenue",
         "CHECK # 131":"United States Treasury",
         "FID BKG SVC LLC MONEYLINE 210402 X49800767 O0TLA JOHN P CLARK":"Transfer from fidelity for tithe",
         "FRANCHISE TAX BD CASTTAXRFD 040521 XXXXX8570 CLARK III, JOHN P & HA":"Franchise tax board CASTAX"}
for idx, row in view_table.iterrows():
    if row["Description"] in notes:
        note = notes[row["Description"]]
        view_table.loc[idx,"comment"] = note
view_table



,Date,Amount,Unk,CheckNum,Description,account_name,spending,income,chase,tithe,taxes,taxes_spend,filtered_amount,filtered_income,filtered_spending,comment
178,2021-03-01,25.00,*,NaN,GO FAR REWARDS,primary,0.00,25.00,0.00,0.00,0.0,0.0,25.00,25.00,0.00,
177,2021-03-01,-100.00,*,NaN,RECURRING TRANSFER TO CLARK J SAVINGS REF #OP09X8TLHP XXXXXX5751,primary,-100.00,0.00,0.00,0.00,0.0,0.0,-100.00,0.00,-100.00,
31,2021-03-01,100.00,*,NaN,RECURRING TRANSFER FROM CLARK J CHECKING REF #OP09X8TLHP XXXXXX9865,savings,0.00,100.00,0.00,0.00,0.0,0.0,100.00,100.00,0.00,
30,2021-03-01,-400.00,*,NaN,RECURRING TRANSFER TO CLARK J REF #OP09XGTWYQ PORTFOLIO CHECKING TO COVER GAS ELECTRIC WATER IN RENTAL,savings,-400.00,0.00,0.00,0.00,0.0,0.0,-400.00,0.00,-400.00,
175,2021-03-01,-534.32,*,NaN,TRANSFER TO CLARK ASTRID ON 03/01 REF #PP09XKGLYW PHONE NOV DEC JAN FEB,primary,-534.32,0.00,0.00,0.00,0.0,0.0,-534.32,0.00,-534.32,
41,2021-03-01,400.00,*,NaN,RECURRING TRANSFER FROM CLARK J REF #OP09XGTWYQ SAVINGS TO COVER GAS ELECTRIC WATER IN RENTAL,umbrella,0.00,400.00,0.00,0.00,0.0,0.0,400.00,400.00,0.00,
176,2021-03-01,-2091.47,*,NaN,RECURRING TRANSFER TO CLARK J REF #OP09XFD2XV CHECKING MORTGAGE PAYMENT,primary,-2091.47,0.00,0.00,0.00,0.0,0.0,-2091.47,0.00,-2091.47,
117,2021-03-01,2091.47,*,NaN,RECURRING TRANSFER FROM CLARK J REF #OP09XFD2XV CHECKING MORTGAGE PAYMENT,utility,0.00,2091.47,0.00,0.00,0.0,0.0,2091.47,2091.47,0.00,
116,2021-03-01,-2091.47,*,NaN,DOVENMUEHLE MTG MORTG PYMT 030121 1465323044 John P Clark III,utility,-2091.47,0.00,0.00,0.00,0.0,0.0,-2091.47,0.00,-2091.47,
115,2021-03-02,-800.00,*,NaN,"RURAL INDIA MINI CONTRIBUTN 210302 XX014I7M91L5NW CLARK, JOHN",utility,-800.00,0.00,0.00,-800.00,0.0,0.0,-800.00,0.00,0.00,


In [ ]:
eoy_primary_spend = end_of_year[end_of_year["Payment Method"].str.startswith("Primary ck acct")]
primary_spend = primary_check[primary_check["Amount"] < 0]

display(eoy_primary_spend.groupby(pd.Grouper(key='Date', freq='1M')).sum("Amount_value"))
display(primary_spend.groupby(pd.Grouper(key='Date', freq='1M')).sum("Amount"))



In [10]:
800 + 600.25 + 600 + 800

2800.25